# Dividing the data into epochs with mne

In [39]:
import os
import mne
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from mne_bids import read_raw_bids, BIDSPath

class BIDSEEGDataset(Dataset):
    def __init__(self, bids_root, patients_tsv):
        self.bids_root = bids_root

        # use only these channels for classification
        self.selected_channels = ['F3','F4','C3','C4','O1','O2','A1','A2']

        # read the patients.tsv file
        self.patients_df = pd.read_csv(patients_tsv, delimiter='\t')

        # create a dictionary with patient_id as key and pathology as value
        self.pathology_dict = dict(zip(self.patients_df['participant_id'], self.patients_df['pathology']))

        # get all the subjects in the bids_root
        self.subjects = [d for d in os.listdir(bids_root) if d.startswith('sub-')] # ['sub-03'] #
        self.all_epochs = []
        self.all_labels = []

        for subject in self.subjects:
            subject_id = subject.replace('sub-', '')
            subject_folder = os.path.join(self.bids_root, subject)
            sessions = [d for d in os.listdir(subject_folder) if d.startswith('ses-')]

            for session in sessions:
                session_id = session.replace('ses-', '')
                bids_path = BIDSPath(subject=subject_id, session=session_id, task='verbalWM',
                                              run='01', datatype='eeg', root=self.bids_root)
                try:
                    raw = read_raw_bids(bids_path, verbose=False)
                except FileNotFoundError:
                    continue
                raw.load_data(verbose=False)
                
                raw.filter(l_freq=1, h_freq=None, verbose=False, fir_design='firwin', skip_by_annotation='edge')
                raw.pick(picks=self.selected_channels, verbose=False)
                raw.resample(sfreq=250)  # Resample to 250 Hz


                events, _ = mne.events_from_annotations(raw, verbose=False)
                epochs = mne.Epochs(raw, events, verbose=False, tmax=8)
                pathology_label = self.pathology_dict.get(f'sub-{subject_id}', 'Unknown')
                for epoch_data in epochs.get_data():
                    self.all_epochs.append(epoch_data)
                    self.all_labels.append(pathology_label)

        # save the labels in a dictionary to convert them to numbers
        # and to easily use them in the dataloader
        self.labels = {}
        for i, pathology in enumerate(set(self.all_labels)):
            self.labels[pathology] = i



    def __len__(self):
        return len(self.all_epochs)

    def __getitem__(self, idx):
        epoch_data = self.all_epochs[idx]
        pathology_label = self.all_labels[idx]
        return torch.tensor(epoch_data, dtype=torch.float32), torch.tensor(self.labels[pathology_label])

In [40]:
from torch.nn.utils.rnn import pad_sequence

# Initialize the custom dataset
bids_root = 'data'
patients_tsv = 'data/participants.tsv'
dataset = BIDSEEGDataset(bids_root, patients_tsv)

# dataset stats
print(f'Total number of samples: {len(dataset)}')

# print number of different pathology labels


/var/folders/zb/vnlk96b52x18jnx35vjsd4f40000gp/T/ipykernel_1842/2084769139.py:36: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = read_raw_bids(bids_path, verbose=False)


Using data from preloaded Raw for 50 events and 2051 original time points ...
2 bad epochs dropped


/var/folders/zb/vnlk96b52x18jnx35vjsd4f40000gp/T/ipykernel_1842/2084769139.py:36: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = read_raw_bids(bids_path, verbose=False)


Using data from preloaded Raw for 50 events and 2051 original time points ...
2 bad epochs dropped


/var/folders/zb/vnlk96b52x18jnx35vjsd4f40000gp/T/ipykernel_1842/2084769139.py:36: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = read_raw_bids(bids_path, verbose=False)


...

Using data from preloaded Raw for 50 events and 2051 original time points ...
2 bad epochs dropped


/var/folders/zb/vnlk96b52x18jnx35vjsd4f40000gp/T/ipykernel_1842/2084769139.py:36: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = read_raw_bids(bids_path, verbose=False)


Using data from preloaded Raw for 50 events and 2051 original time points ...
2 bad epochs dropped


/var/folders/zb/vnlk96b52x18jnx35vjsd4f40000gp/T/ipykernel_1842/2084769139.py:36: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = read_raw_bids(bids_path, verbose=False)


Using data from preloaded Raw for 50 events and 2051 original time points ...
2 bad epochs dropped


In [27]:
# get the distribution of the shape of the samples
shapes = {}
for sample, _ in dataset:
    shape = sample.shape
    if shape in shapes:
        shapes[shape] += 1
    else:
        shapes[shape] = 1

print(f'Distribution of the samples: {shapes}')

Distribution of the samples: {torch.Size([8, 2051]): 3217}


In [28]:
# Create test, validation and train splits
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

# Split the dataset into training and a temporary set (70% train, 30% temp)
# we need this to not loose the `all_labels` list
train_indices, temp_indices = train_test_split(
    list(range(len(dataset))),
    test_size=0.3,
    random_state=42,
    stratify=dataset.all_labels,
    shuffle=True
)

# Split the temporary set into validation and test sets (50% validation, 50% test of the temp set)
val_indices, test_indices = train_test_split(
    temp_indices,
    test_size=0.5,
    random_state=42,
    stratify=[dataset.all_labels[i] for i in temp_indices],
    shuffle=True
)

# Create Subset instances using the split indices
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)


In [29]:
batch_size = 32

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [36]:
from torch import nn
import torch.nn.functional as F

class EEGNet(nn.Module):
    def __init__(self, Chans=8, Samples=2051, dropoutRate=0.5, kernLength=64, F1=8, D=2, F2=16, norm_rate=0.25, dropoutType='Dropout'):
        super(EEGNet, self).__init__()
        
        # First Convolutional Block
        self.block1 = nn.Sequential(
            nn.Conv2d(1, F1, (1, kernLength), padding=(0, kernLength // 2), bias=False),
            nn.BatchNorm2d(F1),
            nn.ELU(),
            nn.AvgPool2d((1, 4)),
            nn.Dropout(dropoutRate)
        )
        
        # Depthwise Convolution
        self.depthwise = nn.Conv2d(F1, F1*D, (Chans, 1), groups=F1, bias=False)
        self.depthwise_bn = nn.BatchNorm2d(F1*D)
        self.depthwise_activation = nn.ELU()
        
        # Separable Convolution
        self.pointwise = nn.Conv2d(F1*D, F2, (1, 1), bias=False)
        self.pointwise_bn = nn.BatchNorm2d(F2)
        self.pointwise_activation = nn.ELU()
        self.pointwise_pooling = nn.AvgPool2d((1, 8))
        self.pointwise_dropout = nn.Dropout(dropoutRate)
        
        # Classification Layer
        self.classify = nn.Linear(F2, 1)
        
    def forward(self, x):
        x = self.block1(x)
        
        # Depthwise Convolution
        x = self.depthwise(x)
        x = self.depthwise_bn(x)
        x = self.depthwise_activation(x)
        
        # Separable Convolution
        x = self.pointwise(x)
        x = self.pointwise_bn(x)
        x = self.pointwise_activation(x)
        x = self.pointwise_pooling(x)
        x = self.pointwise_dropout(x)
        
        x = F.adaptive_avg_pool2d(x, (1, 1)).squeeze()
        x = self.classify(x)
        return x

In [37]:
from torch import optim

net = EEGNet()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [38]:
import torch
import torch.optim as optim

# Hyperparameters
learning_rate = 0.001
num_epochs = 50
device = torch.device('mps') #"cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model, criterion, and optimizer
model = EEGNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {100.*correct/total:.2f}%")

# Testing loop
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for data, labels in test_loader:
        data, labels = data.to(device), labels.to(device)
        outputs = model(data)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

print(f"Test Accuracy: {100.*correct/total:.2f}%")

RuntimeError: Given groups=1, weight of size [8, 1, 1, 64], expected input[1, 32, 8, 2051] to have 1 channels, but got 32 channels instead